In [1]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using ProfileView
using Revise

In [2]:
include("../src/pedestrian_crossing/frenet_pedestrian_emergency_system.jl")

getObjectConfidenceInterval (generic function with 1 method)

In [4]:
params = CrosswalkParams()
params.obstacles_visible = true

obstacle_1 = ConvexPolygon([VecE2(15, -1.5), VecE2(15, -4.5), VecE2(21.5, -4.5), VecE2(21.5, -1.5)],4)
obstacle_2 = ConvexPolygon([VecE2(15, +4.5), VecE2(15, +7.5), VecE2(21.5, +7.5), VecE2(21.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params);



ego_v = 10.0
ped_v = 5.0/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 25.5
ped_y_start = -2
ego_y = 0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;


# Car definition
car_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
car = Vehicle(car_initial_state, VehicleDef(), 1)

# Pedestrian definition using our new Vehicle definition
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

scene = Scene()
push!(scene, car)
push!(scene, ped)



cam = FitToContentCamera(0.);
timestep = 0.05

pos_noise = 0.1
vel_noise = 0.1
false_positive_rate = 0.05
false_negative_rate = 0.01
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

ego_id = 1
ped_id = 2
models[ego_id] = EmergencySystem(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor, obstacles=env.obstacles, timestep=timestep)
models[ped_id] = ConstantPedestrian(v_desired=ped_v, dawdling_amp=0.05) # dumb model


nticks = 80
rec = SceneRecord(nticks+1, timestep)
# execute the simulation

risk = Float64[]
collision_rate = Float64[]
ttc = Float64[]
brake_request = Bool[]
prediction = Vector{Array{Float64}}()

sensor_observations = [Vehicle[]]

obs_callback = (ObservationCallback(risk, collision_rate, ttc, brake_request,prediction,sensor_observations),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()


TTB: 0.885
standby
TTB: 0.885
standby
TTB: 0.885 collision_rate: 0.7 ttc_m: 1.3708564582228597 ttc_std: 0.056802884599380335Risk: 0.6734885226632186 ttc_min: 1.3140535736234793

standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.3421991762168541 ttc_std: 0.049142622002026085Risk: 0.6844248204885294 ttc_min: 1.293056554214828

standby
TTB: 0.885 collision_rate: 0.7 ttc_m: 1.2692139123574324 ttc_std: 0.05279154547838008Risk: 0.7275433468644815 ttc_min: 1.2164223668790524

standby
TTB: 0.885 collision_rate: 0.8 ttc_m: 1.2113301584076364 ttc_std: 0.051347397481153405Risk: 0.7629423727755634 ttc_min: 1.159982760926483

standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.1716721056186148 ttc_std: 0.0457090763599631Risk: 0.785993835501593 ttc_min: 1.1259630292586518

standby
TTB: 0.885 collision_rate: 0.8 ttc_m: 1.1100662027124109 ttc_std: 0.04864174961490225Risk: 0.833785200084041 ttc_min: 1.0614244530975085

standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.0351808741589577 ttc_std: 0.04270472392315

0.495513198

In [5]:
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, prediction)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpSQuHwQ", 0x0000000000000050, 20.0, nothing)